# 데이터셋 검증 및 정리

YOLO 데이터셋 검증 및 캐시 정리

**원본 스크립트**: `check_and_clean_yolo_cls_fine.sh`


## 개요
YOLO 데이터셋 검증 및 캐시 정리

### 기능
- Ultralytics .cache 삭제
- 샘플 파일 존재 여부 확인
- train/val/test 각각 클래스별 이미지 개수 요약


In [ ]:
#!/usr/bin/env bash
set -euo pipefail
DATA_ROOT="/home/wsm/workspace/hit-archlens-project/dataset/icons/yolo_cls_fine"
echo "[1] Ultralytics 캐시 파일 삭제"
if find "$DATA_ROOT" -type f -name "*.cache" | grep -q .; then
  find "$DATA_ROOT" -type f -name "*.cache" -print -delete
else
  echo "  삭제할 .cache 파일 없음"
fi
echo
echo "[2] 에러에 등장한 샘플 파일 존재 여부 확인"
SAMPLE="/home/wsm/workspace/hit-archlens-project/dataset/icons/yolo_cls_fine/train/23/train_176_Res_Amazon-OpenSearch-Service_OpenSearch-Dashboards_48.png"
if [[ -e "$SAMPLE" ]]; then
  echo "  샘플 파일 존재: $SAMPLE"
else
  echo "  샘플 파일 없음 (로그의 No such file과 일치):"
  echo "  $SAMPLE"
fi
echo
echo "[3] split(train/val/test) + 클래스별 이미지 개수 요약"
for SPLIT in train val test; do
  SPLIT_DIR="$DATA_ROOT/$SPLIT"
  if [[ -d "$SPLIT_DIR" ]]; then
    echo "=== $SPLIT ==="
    find "$SPLIT_DIR" -mindepth 1 -maxdepth 1 -type d | sort | while read -r cls_dir; do
      cls_name="$(basename "$cls_dir")"
      cnt=$(find "$cls_dir" -type f \( -iname "*.png" -o -iname "*.jpg" -o -iname "*.jpeg" \) | wc -l)
      printf "%-10s\t%6d files\n" "$cls_name" "$cnt"
    done
    echo
  fi
done
echo "[4] 이제 다시 YOLO 학습을 실행하세요:"
echo "    uv run yolo classify train data=... model=... imgsz=... epochs=..."
